### Unit 1 Homework:  Scraping the Yelp Website

Welcome!  For this homework assignment you'll be tasked with building a web scraper in a manner that builds on what was covered in our web scraping class.

The assignment will extend the lab work done during that time, where we built a dataset that listed the name, number of reviews and price range for restaurant on the following web page: https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1

**What You'll Turn In:**

A finished jupyter notebook that walks us through the steps you took in order to get your results.  Provide notes where appropriate to explain what you are doing.

The notebook should produce a finished dataset at the end.  

If for some reason you're experiencing problems with the final result, please let someone know when turning it in.
 
Your homework will be divided into three tiers, each of which have increasing levels of difficulty:

##### Tier 1: Five Columns From the First Page

At the most basic level for this assignment, you will need to extend what we did in class, and create a dataset that has five columns in it that are 30 rows long.  This means you will not need to go off the first page in order to complete this section.

##### Tier 2:  100 Row Dataset With At Least 3 Columns

For this portion of the assignment, take 3 of your columns from step 1, and extend them out to multiple pages on the yelp website.  You should appropriately account for the presence of missing values.

##### Tier 3:  100 Row Dataset With At Least 5 Columns

Very similar to Tier 2, but if you use this many columns you will be forced to encounter some columns that will frequently have missing values, whereas with Tier 2 you could likely skip these if you wanted to.  

##### Tier 4:  100 Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Restaurants often have different categories associated with them, so grabbing them individually as separate values is often challenging.  To complete this tier, you'll have to find a way to 'pick out' each of the individual categories as their own separate column value.  

##### Tier 5:  Unlimited Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Take what you did in Tier 4, and extend it so that the code will work with an arbitrary number of pages.  Ie, regardless of how many pages there are listing the best restaurants in London, your scraper will find them, and cleanly parse their information into clean datasets.

### Hints

Here are a few tips that will save you time when completing this assignment:

 - The name, average rating, total ratings and neighborhood of a restaurant tend to be the 'easy' ones, because they rarely have missing values, so what ever logic you use on the first page will typically apply to all pages.  They are a good place to start
 - Phone numbers, price ranges and reviews are more commonly missing, so if you are trying to get a larger number of items from them across multiple pages you should expect to do some error handling
 - You can specify any sort of selector when using the `find_all()` method, not just `class`.  For example, imagine you have the following `<div>` tag:
    `<div class='main-container red-blue-green' role='front-unit' aria-select='left-below'>Some content here</div>`
    
   This means that when you use `scraper.find_all('div')`, you can pass in arguments like `scraper.find_all('div', {'role': 'front-unit'})` or anything else that allows you to isolate that particular tag.
 - When specifying selectors like `{'class': 'dkght__384Ko'}`, sometimes less is more.  If you include multiple selectors, you are saying return a tag with **any one of these** distinctions, not all of them.  So if your results are large, try different combinations of selectors to get the smallest results possible.
 - If you begin dealing with values that are unreliably entered, you should use the 'outside in' technique where you grab a parent container that holds the element and find a way to check to see if a particular value is there by scraping it further.  The best way to do this is to try and find a unique container for every single restaurant.  This means that you will have a reliable parent element for every single restaurant, and within *each of these* you can search for `<p>`, `<a>`, `<div>`, and `<span>` tags and apply further logic.
 - When you get results from `BeautifulSoup`, you will be given data that's denoted as either `bs4.element.Tag` or `bs4.element.ResultSet`.  They are **not the same**.  Critically, you can search a `bs4.element.Tag` for further items, but you cannot do this with a `bs4.element.ResultSet`.  
 
   For example, let's say you grab all of the divs from a page with `scraper.find_all('div')` and save it as the variable `total_divs`.  This means `total_divs` will look somethig like this:  
   
   `[<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>]`
      
   In this case the variable `total_divs` is a result set and there's nothing else you can do to it directly.  However, every item within `total_divs` is a tag, which means you can scrape it further.  
   
   So if you wanted you could write a line like:  `total_paragraphs = [div.find_all('p') for div in total_divs]`, and get the collection of paragraphs within each div.  
   
   If you confuse the two you'll get the following error message:  
   
   `AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?`
 - The values of the different selectors change periodically on yelp, so if your scraper all of a sudden stops working that's probably why.  Ie, if you have a command like `scraper.find_all('div', {'class': '485dk0W__container09'}` that no longer returns results, the class `485dk0W__container09` may now be `r56kW__container14` or something similar.

### Testing web scraping skills with a simple website:

In [140]:
candy_url = 'https://www.pd.net/'
candy_page = requests.get(candy_url)
candy_soup = BeautifulSoup(candy_page.content, 'html.parser')

names = candy_soup.find_all('div', class_="node-product")
products = []
for name in names:
    products.append([item for item in (name.text).split('\n') if item])
    
products

[[' A Candy Feast 2020 Pre-order', '$5.99'],
 [' Halloween 2020 Assortment Pre-order', '$5.99'],
 [' Pumpkin Spice', '$5.99'],
 [' Travel The World In Candy Assortment version 3', '$5.99'],
 [' Kiwi Candy', '$5.99'],
 [' Candy Cap Mushroom Candy', '$6.99'],
 [' Horehound & Marshmallow Root Cough Drops', '$5.99'],
 [' Honey and Lavender Cough Drops', '$5.99'],
 [' Violets Are Blue, Crystal Violet Candy', '$5.99'],
 [' Crystal Rose Candy', '$5.99'],
 [' Cinnamon Hearts', '$5.99'],
 [' Watermelon Image Candy', '$5.99'],
 [' Citrus Mix Image Candy', '$5.99'],
 [' Drop Candy Berry Mix', '$5.99'],
 [' Coffee Image candy', '$5.99'],
 [' Grape Drop Candy', '$5.99']]

# Yelp Reviews Code

In [56]:
# your code here
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import math
import numpy as np

yelp_url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1'




In [62]:
def find_yelp_reviews(restaurant_data):
    restaurants = []
    for restaurant in restaurant_data:
        d = dict()
        categories = []
        name = restaurant.find('a', class_='lemon--a__09f24__IEZFH link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95')
        try:
            name = name.text
        except AttributeError:
            name = False
        if name:
            #only do further data work if the restaurant has a valid name; otherwise skip to the next restaurant in the data set
            
            #checking for null values of category so the code doesn't blow up; skips if no valid category
            categories_data = restaurant.find_all('a', class_='lemon--a__09f24__IEZFH link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--default__09f24__3xWLF')
            for data in categories_data:
                try:
                    categories.append(data.text)
                except AttributeError:
                    pass

            # trying to generate a cost rating
            cost = restaurant.find('span', class_="lemon--span__09f24__3997G text__09f24__2tZKC priceRange__09f24__2O6le text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0 text-bullet--after__09f24__1MWoX")
            try:
                cost = str(len(cost.text))
            except AttributeError:
                cost = ''
                
            # trying to generate number of reviews
            num_reviews = restaurant.find('span', class_="lemon--span__09f24__3997G text__09f24__2tZKC reviewCount__09f24__EUXPN text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0")
            try:
                num_reviews = int(num_reviews.text)
            except AttributeError:
                num_reviews = 0
                
            # trying to generate a rating
            rating = restaurant.find('div', class_=re.compile("i-stars"))
            try:
                rating = float(rating['aria-label'].split()[0])
            except AttributeError:
                rating = 0
            except TypeError:
                rating = 0
            #building out a dictionary with the restaurant's attributes
            d['Name'] = name
            d['Rating'] = rating
            d['Cost'] = cost
            d['Reviews'] = num_reviews
            
            # building new columns for each category:
            for i in range(0,len(categories)):
                d[f"Category {i+1}"] = categories[i]
            
            
            # adding completed dictionary for the current page
            restaurants.append(d)
    
    return restaurants
    
def display_yelp_reviews(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Find the total number of pages to determine how many times to loop to get data. Loop should end at (n-1)*10
    num_pages = soup.find('span', class_="lemon--span__09f24__3997G text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0")
    max_start = ((int(num_pages.text.split()[2]))-1)*10
        
    restaurants = []
    
    for index in range(0,max_start+1,10):
        current_page = requests.get(url+'&start='+str(index))
        current_soup = BeautifulSoup(current_page.content, 'html.parser')
        restaurant_data = current_soup.find_all('li', class_='lemon--li__09f24__1r9wz border-color--default__09f24__R1nRO')
        
        restaurants += find_yelp_reviews(restaurant_data)
    
    
    return pd.DataFrame(restaurants)
    
    





In [60]:
london_data = display_yelp_reviews(yelp_url)


{'Name': 'Ffiona’s Restaurant', 'Rating': 4.5, 'Cost': '2', 'Reviews': 269, 'Category 1': 'British'}
{'Name': 'The Mayfair Chippy', 'Rating': 4.5, 'Cost': '2', 'Reviews': 279, 'Category 1': 'Fish & Chips'}
{'Name': 'Duck & Waffle', 'Rating': 4.0, 'Cost': '3', 'Reviews': 703, 'Category 1': 'Modern European', 'Category 2': 'Bars', 'Category 3': 'British'}
{'Name': 'Dishoom', 'Rating': 4.5, 'Cost': '2', 'Reviews': 1841, 'Category 1': 'Indian'}
{'Name': 'The Golden Chippy', 'Rating': 5.0, 'Cost': '2', 'Reviews': 108, 'Category 1': 'Fish & Chips'}
{'Name': 'Restaurant Gordon Ramsay', 'Rating': 4.5, 'Cost': '4', 'Reviews': 205, 'Category 1': 'French', 'Category 2': 'British'}
{'Name': 'Mother Mash', 'Rating': 4.0, 'Cost': '2', 'Reviews': 468, 'Category 1': 'British'}
{'Name': 'Padella', 'Rating': 4.5, 'Cost': '2', 'Reviews': 205, 'Category 1': 'Italian'}
{'Name': 'Flat Iron', 'Rating': 4.5, 'Cost': '2', 'Reviews': 380, 'Category 1': 'Steakhouses'}
{'Name': 'The Queens Arms', 'Rating': 4.5, '

In [64]:
pd.set_option('display.max_rows', None)
# removing NaN values from the category columns and sorting based on rating
london_data = london_data.replace({np.nan: None})
london_data.sort_values(by=['Rating'], ascending=False)

,Name,Rating,Cost,Reviews,Category 1,Category 2,Category 3
97,Bento Bab,5.0,1,10,Korean,None,None
209,The Grill,5.0,,4,British,None,None
115,Trinidad Roti Shop Carribean Cusine,5.0,1,4,Caribbean,None,None
27,Balaio Brazilian Grill,5.0,,1,Brazilian,None,None
149,Cafe Vienna,5.0,2,3,Austrian,None,None
118,Roast Hog,5.0,1,10,Portuguese,Food Stands,None
119,Scully,5.0,,13,British,None,None
188,Madera at Treehouse London,5.0,,1,Mexican,None,None
130,La Mian & Dim Sum Stall,5.0,1,6,Street Vendors,Dim Sum,None
131,Warisan Cafe,5.0,,4,Malaysian,None,None


In [65]:
# london_data.to_csv(r'C:\Users\chloe\Data Science\DAT-1019-Chloe\Data_Sets\london_yelp.csv', index = False, header=True, encoding='utf-8-sig')

In [66]:
gaithersburg_data = display_yelp_reviews('https://www.yelp.com/search?find_desc=Restaurants&find_loc=Gaithersburg%2C%20Maryland&ns=1')
gaithersburg_data = gaithersburg_data.replace({np.nan: None})

In [67]:
# gaithersburg_data.to_csv(r'C:\Users\chloe\Data Science\DAT-1019-Chloe\Data_Sets\gaithersburg_yelp.csv', index = False, header=True, encoding='utf-8-sig')